# Tape Analysis

In [124]:
import pandas as pd
import datetime


In [125]:
df1 = pd.read_excel('input/cvx.xlsx') #import data from excel file
df1.sort_index(ascending=False, inplace = True) # rotate dataFrame
df1.drop(columns=['Bid','Ask','Sizes'],inplace=True) # delete columns
df1.dropna(how='any',inplace=True) # delete Nan Values

In [126]:
df1.index=df1['Date'] # set index as Date column

In [127]:
vol_filter =1000

In [128]:
res_list =[]
pr = 0
dt = 0
tm = 0
sz = 0

ls1=[]

for index, row in df1.iterrows():  # itter every row in order to agregate size
    if (row['Price']== pr) & (row['Time']==tm) :
        sz += row['Size']
        
    if (row['Price']!= pr):
        
        ls1=(dt,tm,pr,sz)
        res_list.append(ls1)
        
        dt = row['Date']
        tm = row['Time']
        pr = row['Price']
        sz = row['Size']


In [129]:
# forming result dataSet
df_output = pd.DataFrame(res_list[1:])
df_output.columns= ['Date','Time','Price','Volume']
df_output['Price_1']=0
df_output['Volume_1']=0
df_output['Vol_filter']=0


In [130]:
pr = 0

for index, row in df_output.iterrows():
    if row['Price']> pr:
        
        df_output.at[index,'Price_1']=row['Price']
        df_output.at[index,'Volume_1']=row['Volume']
      
        
    if row['Price']< pr:
        df_output.at[index,'Price_1']=row['Price']*-1
        df_output.at[index,'Volume_1']=row['Volume']*-1
    pr = row['Price']   
    
    if row['Volume_1']>=vol_filter:
        df_output.at[index,'Vol_filter']= row['Volume_1']


In [131]:
# filtering volume
for index, row in df_output.iterrows():
    if row['Volume_1']>=vol_filter:
        df_output.at[index,'Vol_filter']=row['Volume_1']
    if row['Volume_1']<=vol_filter*-1:
        df_output.at[index,'Vol_filter']=row['Volume_1']    
   
    

In [132]:
df_output.loc['Total'] = df_output.sum(numeric_only=True) # add Total substring



In [133]:
with pd.ExcelWriter("output/output.xlsx") as writer:
   
    df_output.to_excel(writer, sheet_name="output")
    